In [2]:
import pandas as pd
data = pd.read_csv('./3000_8_24.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   publish_time   3000 non-null   object 
 1   view_counts    3000 non-null   object 
 2   summarization  3000 non-null   object 
 3   X              3000 non-null   float64
dtypes: float64(1), object(3)
memory usage: 93.9+ KB


In [4]:
import warnings
warnings.filterwarnings('ignore')
first_column = data.columns[2]
char_to_add = 'Y'
data[first_column] = data[first_column].astype(str) + char_to_add
data.head()

,publish_time,view_counts,summarization,X
0,2011-12-19 15:05:09,20303,黄金白银最新技术分析继续看空黄金任然处于下跌的趋势。Y,0.0
1,2011-12-19 15:25:40,655,周四(1月1日)黄金周线图跌势已缓Y,1.0
2,2011-12-20 03:53:48,1547,想慢慢成才听太公的想快速致富听一亿的我是相信一亿在几天前说1610反弹到位观点果然昨晚到了十...,1.0
3,2011-12-20 07:19:40,1330,黄金从400到320跌幅20 白银纸从10块跌到6快跌幅40 大家都以为白银要到8块包括我在...,0.0
4,2011-12-20 07:43:38,925,银品金屋12月20日早间黄金行情分析分析Y,0.0


In [6]:
data['summarization'] = data['summarization'].astype(str)
s = ' '.join(list(data['summarization']))
s = s.replace(" ", "")
import re
s = s.split('Y')
import jieba
stop_words = {
    '，', '。', '！', '？', '、', '；', '：', '（', '）', '【', '】', '『', '』', '《', '》', '“', '”', '‘', '’', '—', '…', '～'
}
processed_texts = []
for text in s:
    words = jieba.cut(text)  # 分词
    filtered_words = [word for word in words if word not in stop_words and word.strip()]  # 过滤停用词和空字符串
    processed_texts.append(filtered_words)
flattened_list = []
for sublist in processed_texts:
    for item in sublist:
        flattened_list.append(item)
n= list(range(0,len(flattened_list)))
dic = dict(zip(flattened_list,n))
for i in range(0,3000):
    if pd.isna(data.loc[i, 'summarization']) is False:
        lst = processed_texts[i]
        lst = [dic[elem] for elem in lst]
        data['summarization'][i] = lst
    else:
        data['summarization'][i] = [4823]
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86155\AppData\Local\Temp\jieba.cache
Loading model cost 0.462 seconds.
Prefix dict has been built successfully.


,publish_time,view_counts,summarization,X
0,2011-12-19 15:05:09,20303,"[76400, 76062, 76347, 75951, 76281, 76218, 747...",0.0
1,2011-12-19 15:25:40,655,"[75965, 76368, 76261, 76395, 76261, 76372, 763...",1.0
2,2011-12-20 03:53:48,1547,"[66172, 54301, 26, 33, 72445, 76404, 66172, 76...",1.0
3,2011-12-20 07:19:40,1330,"[76400, 75744, 54611, 75911, 81, 76399, 76263,...",0.0
4,2011-12-20 07:43:38,925,"[75341, 75342, 76340, 76395, 76263, 76372, 753...",0.0


In [7]:
m = 0
for i in range(len(data)):
    if len(data.loc[i, 'summarization']) > m:
        m = len(data.loc[i, 'summarization'])
m

80

In [10]:
import numpy as np
arr = np.zeros((3000, 80)) 
for i in range(len(data)):
    x = np.array(data.loc[i, 'summarization'])
    x.resize(80,)
    arr[i] = x
arr

array([[7.6400e+04, 7.6062e+04, 7.6347e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.5965e+04, 7.6368e+04, 7.6261e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [6.6172e+04, 5.4301e+04, 2.6000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [7.6365e+04, 7.6366e+04, 7.6423e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.6400e+04, 7.6401e+04, 7.6402e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.6406e+04, 7.6407e+04, 7.6408e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [12]:
np.save('./arr.npy', arr)

In [14]:
import pandas as pd
import keras as keras
import keras.layers as layers
from keras.datasets import imdb
import tensorflow as tf
import os
import numpy as np
#from keras.utils import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [15]:
npword = np.load('./arr.npy')
npword = npword.astype(int)
npword

array([[76400, 76062, 76347, ...,     0,     0,     0],
       [75965, 76368, 76261, ...,     0,     0,     0],
       [66172, 54301,    26, ...,     0,     0,     0],
       ...,
       [76365, 76366, 76423, ...,     0,     0,     0],
       [76400, 76401, 76402, ...,     0,     0,     0],
       [76406, 76407, 76408, ...,     0,     0,     0]])

In [16]:
npword.shape

(3000, 80)

In [20]:
x_train = npword[:2400]
x_test = npword[2400:]

In [22]:
data = pd.read_csv('./3000_8_24.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   publish_time   3000 non-null   object 
 1   view_counts    3000 non-null   object 
 2   summarization  3000 non-null   object 
 3   X              3000 non-null   float64
dtypes: float64(1), object(3)
memory usage: 93.9+ KB


In [24]:
y = np.array(data['X'])
y_train = y[:2400]
y_test = y[2400:]

In [26]:
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [28]:
#得到等长文本
max_word = 80
x_train.shape

(2400, 80)

In [30]:
#获取训练集的总词汇数
vocab_size = np.max([np.max(x_train[i]) for i in range(x_train.shape[0])]) + 1  # 这里1 代表空格，其索引被认为是0。
vocab_size

76435

In [32]:
from tensorflow.keras.optimizers import Adam
#构建模型
inputs = keras.Input(shape=(80,), name='img')
x = keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_word)(inputs)
x = keras.layers.Reshape((80, 64, 1), input_shape=(80, 64))(x)

x1 = keras.layers.Conv2D(filters=32, kernel_size=(5, 32), padding='valid', activation='relu')(x)
#x1 = layers.BatchNormalization()(x1)
x1 = keras.layers.GlobalMaxPool2D()(x1)

x2 = keras.layers.Conv2D(filters=32, kernel_size=(4, 32), padding='valid', activation='relu')(x)
# x2 = layers.BatchNormalization()(x2)
x2 = keras.layers.GlobalMaxPool2D()(x2)

x3 = keras.layers.Conv2D(filters=32, kernel_size=(3, 32), padding='valid', activation='relu')(x)
# x3 = layers.BatchNormalization()(x3)
x3 = keras.layers.GlobalMaxPool2D()(x3)

x = layers.Concatenate(axis=1)([x1, x2, x3])
#x = layers.Concatenate(axis=1)([x1, x2])

outputs = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='txt_cnn')

model.compile(optimizer=Adam(),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])

model.summary()

Model: "txt_cnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 80, 64)       4891840     img[0][0]                        
__________________________________________________________________________________________________
reshape (Reshape)               (None, 80, 64, 1)    0           embedding[0][0]                  
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 76, 33, 32)   5152        reshape[0][0]                    
____________________________________________________________________________________________

In [34]:
model.fit(x_train, y_train,
            validation_data=(x_test, y_test),
            epochs=10,
            batch_size=64,
            verbose=1)

Epoch 1/10
38/38 [==============================] - 5s 30ms/step - loss: 0.6818 - categorical_accuracy: 0.5725 - val_loss: 0.6372 - val_categorical_accuracy: 0.6933
Epoch 2/10
38/38 [==============================] - 1s 14ms/step - loss: 0.6709 - categorical_accuracy: 0.5729 - val_loss: 0.6482 - val_categorical_accuracy: 0.6933
Epoch 3/10
38/38 [==============================] - 1s 14ms/step - loss: 0.6347 - categorical_accuracy: 0.6187 - val_loss: 0.6189 - val_categorical_accuracy: 0.7583
Epoch 4/10
38/38 [==============================] - 1s 14ms/step - loss: 0.5040 - categorical_accuracy: 0.7846 - val_loss: 0.5112 - val_categorical_accuracy: 0.7817
Epoch 5/10
38/38 [==============================] - 1s 14ms/step - loss: 0.3458 - categorical_accuracy: 0.8629 - val_loss: 0.5063 - val_categorical_accuracy: 0.7617
Epoch 6/10
38/38 [==============================] - 1s 14ms/step - loss: 0.2206 - categorical_accuracy: 0.9242 - val_loss: 0.5174 - val_categorical_accuracy: 0.7650
Epoch 7/10

In [36]:
# 对文本进行预测
prediction = model.predict(x_test)
prediction

array([[0.9916842 , 0.00831584],
       [0.98270357, 0.01729643],
       [0.30060074, 0.6993993 ],
       ...,
       [0.9916905 , 0.00830945],
       [0.996268  , 0.00373211],
       [0.8381582 , 0.16184182]], dtype=float32)

In [38]:
prediction0 = model.predict(x_train)
prediction0

array([[0.9954437 , 0.00455626],
       [0.07374924, 0.9262507 ],
       [0.0021871 , 0.99781287],
       ...,
       [0.97287863, 0.02712138],
       [0.9982521 , 0.00174792],
       [0.99864155, 0.0013585 ]], dtype=float32)

In [40]:
y_test[:,0]

array([1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 0.

In [42]:
prediction[:,0]

array([9.91684198e-01, 9.82703567e-01, 3.00600737e-01, 9.23300087e-01,
       7.81020582e-01, 9.76366043e-01, 9.34751034e-01, 9.86223042e-01,
       9.85751808e-01, 3.31375182e-01, 9.93721902e-01, 9.97384250e-01,
       9.67139900e-01, 9.64897990e-01, 9.97320592e-01, 9.55612779e-01,
       7.76541531e-01, 8.45770314e-02, 9.71993148e-01, 8.13725531e-01,
       9.68565047e-01, 7.03348815e-01, 9.77669060e-01, 3.56309503e-01,
       1.09544925e-01, 8.73633400e-02, 9.91948187e-01, 8.15546155e-01,
       5.54735735e-02, 4.20517296e-01, 3.59498300e-02, 9.11124766e-01,
       9.34630096e-01, 2.44980171e-01, 4.36567575e-01, 7.77241349e-01,
       9.91357446e-01, 9.08321381e-01, 7.76871860e-01, 3.70876603e-02,
       1.76542625e-01, 2.71105766e-02, 2.45077744e-01, 8.21496665e-01,
       9.31057155e-01, 9.55549896e-01, 9.34878111e-01, 9.93389010e-01,
       9.99680519e-01, 2.32691064e-01, 5.74542992e-02, 9.89277244e-01,
       9.55430031e-01, 3.51270676e-01, 8.85883510e-01, 3.05417538e-01,
      

In [44]:
pred = np.concatenate((prediction[:,0], prediction0[:,0]))

In [46]:
np.save('./pred.npy', pred)

In [48]:
for i in range(len(prediction)):
    if prediction[i,0] > 0.5:
        prediction[i,0] = 1
    else:
        prediction[i,0] = 0

In [50]:
prediction[:,0]

array([1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 0.

In [52]:
from sklearn.metrics import classification_report
t = classification_report(y_test[:,0], prediction[:,0])
print(t)

              precision    recall  f1-score   support

         0.0       0.61      0.58      0.60       184
         1.0       0.82      0.84      0.83       416

    accuracy                           0.76       600
   macro avg       0.72      0.71      0.71       600
weighted avg       0.76      0.76      0.76       600



In [54]:
model.save('text_cnn_model.h5')